# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'NVDA',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (NVDA). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for NVDA to the Data Analyst. They will be able to use statistical modeling and machine learning to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for NVDA using statistical modeling and machine learning",
    "context": "The task involves analyzing market data for the selected stock NVDA, using statistical modeling and machine learning techniques to identify trends and predict market movements."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for NVDA to anal

Final Answer: NVIDIA Corporation (NVDA) provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The company's products are used in gaming, professional visualization, data center, and automotive markets. NVDA's stock price has seen significant fluctuations in the past year, with a YTD return of 138.50% and a 1-year return of 179.82%. Key financial highlights include a market cap of 2.90T, a profit margin of 53.40%, and a return on equity of 115.66%. NVDA's performance compared to benchmark S&P 500 indicates strong returns over various time frames. This information provides valuable insights for analyzing market trends and predicting future movements for NVDA.

> Finished chain.
 

NVIDIA Corporation (NVDA) provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The company's products are used in gaming, professional visualization, data center, and auto

Action Input: {"search_query": "NVDA stock news"} 


Search results: Title: Nvidia (NVDA) Stock Trades Down, Here Is Why - Yahoo Finance
Link: https://finance.yahoo.com/news/nvidia-nvda-stock-trades-down-185018096.html
Snippet: Shares of leading designer of graphics chips Nvidia (NASDAQ:NVDA) fell 6.5% in the afternoon session after the stock continued to pull back, ...
---
Title: Nvidia stock falls more than 6% as investors rotate out of chip ...
Link: https://finance.yahoo.com/news/nvidia-stock-falls-more-than-6-as-investors-rotate-out-of-chip-heavyweight-165458081.html
Snippet: Nvidia (NVDA) stock fell more than 6% to close at $118.11 per share on Monday as investors rotated out of the hottest AI play of the year.
---
Title: Nvidia Stock Is in a Correction - Barron's
Link: https://www.barrons.com/articles/nvidia-stock-price-today-de134fa6
Snippet: Nvidia can't hold onto its gains. It fell into a correction Monday and has lost $430 billion in market value since it briefly became the 

I need to gather detailed information on the approved trading strategies for NVDA to determine the best execution methods.

Action: Read website content
Action Input: {"website_url": "https://www.nasdaq.com/market-activity/stocks/nvda"} 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='www.nasdaq.com', port=443): Read timed out. (read timeout=15).
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a website content.

Thought: I should try to gather information on NVDA trading strategies using a different approach.
Action: Search the internet
Action Input: {"search_query": "NVDA approved trading strategies"} 


Search results: Title: NVDA Strategy : r/options - Reddit
Link: https://www.reddit.com/r/options/comments/14yzi78/nvda_strategy/
Snippet: One approach you could take is to buy a LEAP with a low delta (like at the 200 or 250 strike) and just see where that

Action Input: {"search_query": "NVDA stock performance and recent news"} 


Search results: Title: NVIDIA Corporation (NVDA) Latest Stock News & Headlines
Link: https://finance.yahoo.com/quote/NVDA/news/
Snippet: Get the latest NVIDIA Corporation (NVDA) stock news and headlines to help you in your trading and investing decisions.
---
Title: NVIDIA Corp NVDA:NASDAQ - Stock Price, Quote and News - CNBC
Link: https://www.cnbc.com/quotes/NVDA
Snippet: Get NVIDIA Corp (NVDA:NASDAQ) real-time stock quotes, news, price and financial information from CNBC.
---
Title: Nvidia's stock enters correction territory less than a week after hitting ...
Link: https://www.morningstar.com/news/marketwatch/20240624147/nvidias-stock-enters-correction-territory-less-than-a-week-after-hitting-a-high
Snippet: Nvidia shares fell 6.7% in Monday trading, to $118.11. The three-day slide shaved more than $400 billion off the company's market capitalization ...
---
Title: Dow ends 260 points higher, Nvidia slips int

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

As per the analysis conducted, it is essential to recognize the heightened risk associated with trading NVDA, especially considering the recent correction in its stock price. The proposed trading strategies involving buying LEAPs with low deltas and utilizing margin loans for leveraging may pose increased risk due to the current market conditions. It is recommended to exercise caution and consider implementing risk mitigation measures such as stop-loss orders, diversification of the portfolio, and setting strict risk management rules to protect against potential losses. Furthermore, continuous monitoring of market trends, news, and NVDA's performance is crucial to make informed trading decisions and adapt strategies accordingly. Backtesting of the proposed strategies is highly recommended to assess their effectiveness and identify any potential weaknesses. In light of the current market volatility, it is advisable to reevaluate and potentially adjust the trading strategies for NVDA to minimize risk exposure and maximize profitability.